<a href="https://colab.research.google.com/github/neurovolt/Clock/blob/main/CogVideoX_5B_jupyter_free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# ENHANCED WEB INTERFACE WITH COLOR CORRECTION & NGROK TOKEN
# ============================================

!pip install -q diffusers transformers accelerate imageio-ffmpeg safetensors opencv-python flask-ngrok pyngrok

import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler
from diffusers.utils import export_to_video
from flask import Flask, render_template_string, request, send_file, jsonify
import cv2
import numpy as np
import os
from datetime import datetime

# ============================================
# 🔑 SET YOUR NGROK TOKEN HERE
# ============================================
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # ⬅️ PASTE YOUR TOKEN HERE
# Get free token from: https://dashboard.ngrok.com/get-started/your-authtoken
# ============================================

# Initialize Flask app
app = Flask(__name__)

# Global variables
pipe = None
adapter = None

def load_model():
    global pipe, adapter
    if pipe is None:
        print("Loading model...")
        adapter = MotionAdapter.from_pretrained(
            "guoyww/animatediff-motion-adapter-v1-5-2",
            torch_dtype=torch.float16
        )

        model_id = "SG161222/Realistic_Vision_V6.0_B1_noVAE"

        pipe = AnimateDiffPipeline.from_pretrained(
            model_id,
            motion_adapter=adapter,
            torch_dtype=torch.float16
        )
        pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
        pipe.enable_model_cpu_offload()
        pipe.enable_vae_slicing()
        print("✅ Model loaded!")

def fix_color_saturation(frame):
    """Fix over-saturation and color issues"""
    # Convert to LAB color space
    lab = cv2.cvtColor(frame, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Reduce saturation by scaling a and b channels
    a = cv2.multiply(a, 0.85)  # Reduce red-green saturation
    b = cv2.multiply(b, 0.85)  # Reduce blue-yellow saturation

    # Merge back
    lab = cv2.merge([l, a, b])
    corrected = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

    # Apply gentle contrast adjustment (CLAHE)
    lab2 = cv2.cvtColor(corrected, cv2.COLOR_RGB2LAB)
    l2, a2, b2 = cv2.split(lab2)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l2)
    lab2 = cv2.merge([l2, a2, b2])
    final = cv2.cvtColor(lab2, cv2.COLOR_LAB2RGB)

    # Subtle color temperature adjustment (reduce warmth)
    final = final.astype(np.float32)
    final[:, :, 0] = np.clip(final[:, :, 0] * 0.95, 0, 255)  # Reduce red slightly
    final[:, :, 2] = np.clip(final[:, :, 2] * 1.02, 0, 255)  # Increase blue slightly

    return final.astype(np.uint8)

def enhance_video(frames, width, height, enable_upscale=True, enable_interpolation=True, fix_colors=True):
    """Post-process video with upscaling, interpolation, and color correction"""
    processed_frames = []

    for frame in frames:
        frame_np = np.array(frame)

        # Fix color saturation first
        if fix_colors:
            frame_np = fix_color_saturation(frame_np)

        if enable_upscale:
            # Upscale 2x with better algorithm
            upscaled = cv2.resize(frame_np, (width*2, height*2), interpolation=cv2.INTER_LANCZOS4)

            # Gentle sharpening (less aggressive)
            kernel = np.array([[-0.5,-0.5,-0.5],
                              [-0.5, 5,-0.5],
                              [-0.5,-0.5,-0.5]])
            sharpened = cv2.filter2D(upscaled, -1, kernel * 0.15)

            # Reduce noise
            denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 3, 3, 7, 21)
            processed_frames.append(denoised)
        else:
            processed_frames.append(frame_np)

    if enable_interpolation and len(processed_frames) > 1:
        # Interpolate frames for smoother motion
        interpolated_frames = []
        for i in range(len(processed_frames) - 1):
            interpolated_frames.append(processed_frames[i])

            frame1 = processed_frames[i].astype(np.float32)
            frame2 = processed_frames[i + 1].astype(np.float32)

            # Add 2 interpolated frames
            interp1 = (frame1 * 0.66 + frame2 * 0.34).astype(np.uint8)
            interp2 = (frame1 * 0.34 + frame2 * 0.66).astype(np.uint8)

            interpolated_frames.append(interp1)
            interpolated_frames.append(interp2)

        interpolated_frames.append(processed_frames[-1])
        return interpolated_frames, 24  # 24 fps

    return processed_frames, 8  # 8 fps

def generate_video(prompt, negative_prompt, width, height, num_frames, steps, guidance, seed, enable_upscale, enable_interpolation, fix_colors):
    """Generate and enhance video"""
    load_model()

    print(f"🎬 Generating {width}x{height} video...")

    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_frames=num_frames,
        height=height,
        width=width,
        guidance_scale=guidance,
        num_inference_steps=steps,
        generator=torch.Generator("cpu").manual_seed(seed),
    )

    frames = output.frames[0]

    # Enhance video with color correction
    print("📈 Enhancing quality & fixing colors...")
    enhanced_frames, fps = enhance_video(frames, width, height, enable_upscale, enable_interpolation, fix_colors)

    # Save
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"video_{timestamp}.mp4"
    export_to_video(enhanced_frames, filename, fps=fps)

    final_width = width * 2 if enable_upscale else width
    final_height = height * 2 if enable_upscale else height

    return filename, final_width, final_height, fps

# HTML Template
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>AI Video Generator Pro</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            padding: 40px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
        }
        h1 {
            color: #333;
            margin-bottom: 10px;
            text-align: center;
            font-size: 2.5em;
        }
        .subtitle {
            text-align: center;
            color: #666;
            margin-bottom: 30px;
            font-size: 1.1em;
        }
        .form-group {
            margin-bottom: 25px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            color: #555;
            font-weight: 600;
            font-size: 14px;
        }
        textarea, input, select {
            width: 100%;
            padding: 12px;
            border: 2px solid #e0e0e0;
            border-radius: 8px;
            font-size: 14px;
            transition: border 0.3s;
            font-family: inherit;
        }
        textarea:focus, input:focus, select:focus {
            outline: none;
            border-color: #667eea;
        }
        textarea {
            min-height: 100px;
            resize: vertical;
        }
        .settings-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
        }
        .preset-buttons {
            display: flex;
            gap: 10px;
            margin-bottom: 15px;
            flex-wrap: wrap;
        }
        .preset-btn {
            padding: 8px 16px;
            background: #f0f0f0;
            border: 2px solid #ddd;
            border-radius: 6px;
            cursor: pointer;
            font-size: 13px;
            transition: all 0.3s;
        }
        .preset-btn:hover, .preset-btn.active {
            background: #667eea;
            color: white;
            border-color: #667eea;
        }
        .quality-section {
            background: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
            margin-bottom: 20px;
            border: 2px solid #e9ecef;
        }
        .quality-section h3 {
            margin-bottom: 15px;
            color: #333;
        }
        .checkbox-group {
            display: flex;
            gap: 20px;
            flex-wrap: wrap;
        }
        .checkbox-item {
            display: flex;
            align-items: center;
            gap: 8px;
        }
        .checkbox-item input[type="checkbox"] {
            width: auto;
            height: 20px;
            width: 20px;
            cursor: pointer;
        }
        .checkbox-item label {
            margin: 0;
            cursor: pointer;
            font-weight: normal;
        }
        button[type="submit"] {
            width: 100%;
            padding: 18px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            border-radius: 10px;
            font-size: 18px;
            font-weight: 600;
            cursor: pointer;
            transition: transform 0.2s;
        }
        button[type="submit"]:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(102, 126, 234, 0.4);
        }
        button[type="submit"]:disabled {
            background: #ccc;
            cursor: not-allowed;
            transform: none;
        }
        #loading {
            display: none;
            text-align: center;
            margin: 30px 0;
            padding: 30px;
            background: #f8f9fa;
            border-radius: 10px;
        }
        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 60px;
            height: 60px;
            animation: spin 1s linear infinite;
            margin: 0 auto 20px;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        #result {
            margin-top: 30px;
            text-align: center;
        }
        video {
            max-width: 100%;
            border-radius: 10px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
        }
        .video-info {
            background: #e7f3ff;
            padding: 15px;
            border-radius: 8px;
            margin: 15px 0;
            text-align: left;
        }
        .download-btn {
            display: inline-block;
            padding: 12px 30px;
            background: #667eea;
            color: white;
            text-decoration: none;
            border-radius: 8px;
            margin-top: 10px;
            font-weight: 600;
            transition: all 0.3s;
        }
        .download-btn:hover {
            background: #5568d3;
            transform: translateY(-2px);
        }
        .info-box {
            background: #dcfce7;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            border-left: 4px solid #16a34a;
        }
        .warning-box {
            background: #fff3cd;
            padding: 15px;
            border-radius: 8px;
            margin-bottom: 20px;
            border-left: 4px solid #ffc107;
        }
        .tips {
            background: #f0f7ff;
            padding: 20px;
            border-radius: 8px;
            margin-top: 30px;
            border-left: 4px solid #0066cc;
        }
        .tips h3 {
            margin-bottom: 15px;
            color: #0066cc;
        }
        .tips ul {
            margin-left: 20px;
        }
        .tips li {
            margin: 8px 0;
            color: #333;
            line-height: 1.6;
        }
        .example-prompts {
            background: #f0f7ff;
            padding: 20px;
            border-radius: 8px;
            margin-top: 20px;
        }
        .example-prompts h3 {
            margin-bottom: 15px;
            color: #0066cc;
        }
        .example-prompt {
            background: white;
            padding: 12px;
            margin: 10px 0;
            border-radius: 6px;
            cursor: pointer;
            border: 2px solid #e0e0e0;
            transition: all 0.3s;
        }
        .example-prompt:hover {
            border-color: #667eea;
            transform: translateX(5px);
        }
        .example-prompt strong {
            display: block;
            color: #667eea;
            margin-bottom: 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎬 AI Video Generator Pro</h1>
        <p class="subtitle">With Color Correction, Upscaling & Frame Interpolation</p>

        <div class="info-box">
            <strong>✅ Color Fix Applied:</strong> Automatic saturation correction, color balance, and natural skin tones
        </div>

        <div class="warning-box">
            <strong>⚠️ Reduce Saturation in Prompts:</strong> Avoid words like "vibrant", "vivid", "saturated". Use "natural lighting", "soft tones", "realistic colors" instead.
        </div>

        <form id="videoForm">
            <div class="form-group">
                <label>🎨 Prompt (Use NATURAL, REALISTIC descriptions):</label>
                <textarea name="prompt" id="prompt" required>RAW photo, 8k uhd, professional photography, beautiful woman, age 25, long brown hair, white linen dress, walking on beach, soft natural lighting, golden hour, muted tones, realistic colors, detailed face, natural skin tone, photorealistic, film photography</textarea>
            </div>

            <div class="form-group">
                <label>🚫 Negative Prompt (IMPORTANT for color):</label>
                <textarea name="negative_prompt" id="negative_prompt">cartoon, anime, 3d render, oversaturated, vibrant colors, neon, unrealistic colors, painting, illustration, worst quality, low quality, blurry, deformed, mutation, glitch, duplicate, watermark, text</textarea>
            </div>

            <div class="quality-section">
                <h3>✨ Enhancement Options</h3>
                <div class="checkbox-group">
                    <div class="checkbox-item">
                        <input type="checkbox" name="enable_upscale" id="enable_upscale" checked>
                        <label for="enable_upscale">2x Upscaling</label>
                    </div>
                    <div class="checkbox-item">
                        <input type="checkbox" name="enable_interpolation" id="enable_interpolation" checked>
                        <label for="enable_interpolation">Frame Interpolation (24fps)</label>
                    </div>
                    <div class="checkbox-item">
                        <input type="checkbox" name="fix_colors" id="fix_colors" checked>
                        <label for="fix_colors">⭐ Color Correction (Recommended)</label>
                    </div>
                </div>
            </div>

            <div class="form-group">
                <label>📐 Resolution Presets:</label>
                <div class="preset-buttons">
                    <button type="button" class="preset-btn" onclick="setResolution(512, 512)">Square</button>
                    <button type="button" class="preset-btn active" onclick="setResolution(512, 768)">Portrait ⭐</button>
                    <button type="button" class="preset-btn" onclick="setResolution(768, 512)">Landscape</button>
                </div>
            </div>

            <div class="settings-grid">
                <div class="form-group">
                    <label>Width:</label>
                    <input type="number" name="width" id="width" value="512" min="256" max="768" step="64">
                </div>
                <div class="form-group">
                    <label>Height:</label>
                    <input type="number" name="height" id="height" value="768" min="256" max="896" step="64">
                </div>
                <div class="form-group">
                    <label>Frames:</label>
                    <input type="number" name="num_frames" value="24" min="8" max="32" step="8">
                </div>
                <div class="form-group">
                    <label>Steps:</label>
                    <input type="number" name="steps" value="30" min="20" max="50" step="5">
                </div>
                <div class="form-group">
                    <label>Guidance (Lower = Less saturated):</label>
                    <input type="number" name="guidance" value="6.5" min="5" max="10" step="0.5">
                </div>
                <div class="form-group">
                    <label>Seed:</label>
                    <input type="number" name="seed" value="42" min="0">
                </div>
            </div>

            <button type="submit" id="generateBtn">🎬 Generate Natural Video</button>
        </form>

        <div id="loading">
            <div class="spinner"></div>
            <h3>🎬 Generating video...</h3>
            <p id="statusText">8-15 minutes • Applying color correction...</p>
        </div>

        <div id="result"></div>

        <div class="example-prompts">
            <h3>💡 Natural Color Prompts (Click to use)</h3>

            <div class="example-prompt" onclick="usePrompt(0)">
                <strong>🌅 Beach Natural</strong>
                <span>Soft natural lighting, muted tones, realistic colors</span>
            </div>

            <div class="example-prompt" onclick="usePrompt(1)">
                <strong>🌲 Forest Muted</strong>
                <span>Overcast lighting, natural greens, film photography look</span>
            </div>

            <div class="example-prompt" onclick="usePrompt(2)">
                <strong>👤 Portrait Natural</strong>
                <span>Studio lighting, natural skin tones, professional photography</span>
            </div>
        </div>

        <div class="tips">
            <h3>🎨 Tips to Avoid Over-Saturation:</h3>
            <ul>
                <li><strong>Always use:</strong> "natural lighting", "soft tones", "muted colors", "realistic"</li>
                <li><strong>Never use:</strong> "vibrant", "vivid", "neon", "saturated", "colorful"</li>
                <li><strong>Add to negative:</strong> "oversaturated, vibrant colors, neon"</li>
                <li><strong>Lower guidance scale:</strong> Use 6-7 instead of 7.5-8</li>
                <li><strong>Enable color correction:</strong> Always keep it checked!</li>
                <li><strong>Lighting keywords:</strong> "soft lighting", "natural daylight", "golden hour", "overcast"</li>
                <li><strong>For skin tones:</strong> Add "natural skin tone, realistic complexion"</li>
            </ul>
        </div>
    </div>

    <script>
        const examplePrompts = [
            {
                prompt: "RAW photo, 8k uhd, beautiful woman, age 25, long hair, white dress, walking on beach, soft natural lighting, golden hour, muted warm tones, realistic colors, detailed face, natural skin tone, photorealistic, film photography, slight grain",
                negative: "cartoon, anime, oversaturated, vibrant colors, neon, unrealistic colors, worst quality, low quality, blurry, deformed, glitch"
            },
            {
                prompt: "RAW photo, woman in red coat, walking through autumn forest, overcast lighting, muted natural greens and browns, soft diffused light, realistic colors, natural skin tone, photorealistic, film photography aesthetic",
                negative: "cartoon, anime, oversaturated, vibrant colors, neon, unrealistic colors, worst quality, low quality, blurry, deformed, glitch"
            },
            {
                prompt: "RAW photo, 8k uhd, close-up portrait, beautiful woman, age 27, natural makeup, soft studio lighting, neutral background, natural skin tone, realistic complexion, professional photography, muted colors, photorealistic",
                negative: "cartoon, anime, oversaturated, vibrant colors, neon, unrealistic colors, worst quality, low quality, blurry, deformed, glitch"
            }
        ];

        function usePrompt(index) {
            document.getElementById('prompt').value = examplePrompts[index].prompt;
            document.getElementById('negative_prompt').value = examplePrompts[index].negative;
        }

        function setResolution(width, height) {
            document.getElementById('width').value = width;
            document.getElementById('height').value = height;
            document.querySelectorAll('.preset-btn').forEach(btn => btn.classList.remove('active'));
            event.target.classList.add('active');
        }

        document.getElementById('videoForm').onsubmit = async function(e) {
            e.preventDefault();

            const btn = document.getElementById('generateBtn');
            const loading = document.getElementById('loading');
            const result = document.getElementById('result');

            btn.disabled = true;
            btn.textContent = '⏳ Generating...';
            loading.style.display = 'block';
            result.innerHTML = '';

            const formData = new FormData(this);
            const params = {};

            for (let [key, value] of formData.entries()) {
                if (key === 'enable_upscale' || key === 'enable_interpolation' || key === 'fix_colors') {
                    params[key] = true;
                } else {
                    params[key] = value;
                }
            }

            if (!formData.has('enable_upscale')) params['enable_upscale'] = false;
            if (!formData.has('enable_interpolation')) params['enable_interpolation'] = false;
            if (!formData.has('fix_colors')) params['fix_colors'] = false;

            try {
                const response = await fetch('/generate', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(params)
                });

                if (response.ok) {
                    const data = await response.json();
                    loading.style.display = 'none';

                    result.innerHTML = `
                        <h2>✅ Natural Video Generated!</h2>

                        <div class="video-info">
                            <strong>📊 Details:</strong><br>
                            Resolution: ${data.width}x${data.height}<br>
                            FPS: ${data.fps}<br>
                            Color Corrected: ${data.color_fixed ? '✅ Yes' : '❌ No'}
                        </div>

                        <video controls autoplay loop>
                            <source src="/video/${data.filename}" type="video/mp4">
                        </video>

                        <a href="/video/${data.filename}" download class="download-btn">
                            📥 Download Video
                        </a>

                        <p style="margin-top: 20px; color: #666;">
                            Runtime will restart in 10 seconds...
                        </p>
                    `;

                    setTimeout(() => {
                        fetch('/restart', { method: 'POST' });
                    }, 10000);
                } else {
                    throw new Error('Generation failed');
                }
            } catch (error) {
                loading.style.display = 'none';
                result.innerHTML = `<p style="color: red;">❌ Error: ${error.message}</p>`;
            }

            btn.disabled = false;
            btn.textContent = '🎬 Generate Natural Video';
        };
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/generate', methods=['POST'])
def generate():
    try:
        data = request.json

        enable_upscale = data.get('enable_upscale', True)
        enable_interpolation = data.get('enable_interpolation', True)
        fix_colors = data.get('fix_colors', True)

        filename, width, height, fps = generate_video(
            prompt=data['prompt'],
            negative_prompt=data['negative_prompt'],
            width=int(data['width']),
            height=int(data['height']),
            num_frames=int(data['num_frames']),
            steps=int(data['steps']),
            guidance=float(data['guidance']),
            seed=int(data['seed']),
            enable_upscale=enable_upscale,
            enable_interpolation=enable_interpolation,
            fix_colors=fix_colors
        )

        return jsonify({
            'success': True,
            'filename': filename,
            'width': width,
            'height': height,
            'fps': fps,
            'color_fixed': fix_colors
        })
    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/video/<filename>')
def serve_video(filename):
    return send_file(filename, mimetype='video/mp4')

@app.route('/restart', methods=['POST'])
def restart():
    import os
    os.kill(os.getpid(), 9)
    return jsonify({'success': True})

# Setup ngrok with token
from pyngrok import ngrok, conf

if NGROK_TOKEN != "YOUR_NGROK_TOKEN_HERE":
    conf.get_default().auth_token = NGROK_TOKEN
    print("✅ Ngrok token set!")
else:
    print("⚠️  No ngrok token set. Get one from: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   The tunnel may expire after 2 hours without a token.\n")

ngrok.kill()
public_url = ngrok.connect(5000)

print("\n" + "="*70)
print("🚀 ENHANCED WEB INTERFACE WITH COLOR FIX")
print("="*70)
print(f"📱 Open this URL:\n   {public_url}")
print("="*70)
print("\n✨ Color Fixes Applied:")
print("   • Automatic saturation reduction")
print("   • Color temperature correction")
print("   • Natural skin tone adjustment")
print("   • Contrast normalization")
print("\n⏱️  Time: 8-15 minutes")
print("✅ Keep running!\n")

app.run(port=5000)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


⚠️  No ngrok token set. Get one from: https://dashboard.ngrok.com/get-started/your-authtoken
   The tunnel may expire after 2 hours without a token.


🚀 ENHANCED WEB INTERFACE WITH COLOR FIX
📱 Open this URL:
   NgrokTunnel: "https://margot-stylish-norene.ngrok-free.dev" -> "http://localhost:5000"

✨ Color Fixes Applied:
   • Automatic saturation reduction
   • Color temperature correction
   • Natural skin tone adjustment
   • Contrast normalization

⏱️  Time: 8-15 minutes
✅ Keep running!

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Feb/2026 16:05:26] "GET / HTTP/1.1" 200 -


Loading model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your confi

...

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--SG161222--Realistic_Vision_V6.0_B1_noVAE/snapshots/9a857a696b9aabbf509073e0aa55ec8200b6ef7d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
CLIPFeatureExtractor appears to have been deprecated in transformers. U

✅ Model loaded!
🎬 Generating 512x768 video...


  0%|          | 0/30 [00:00<?, ?it/s]

📈 Enhancing quality & fixing colors...


INFO:werkzeug:127.0.0.1 - - [10/Feb/2026 16:08:16] "POST /generate HTTP/1.1" 200 -
